In [ ]:
import os 
import multiprocessing
import itertools
import pandas as pd
import numpy as np
import lib.helpers as hlp
import lib.backtest as backtest
import lib.plotting as plotting


In [ ]:
# all backtest settings
historical_df = hlp.get_historical_prices_df()
input_df = pd.read_csv(os.path.join(hlp.INPUT_DATA_DIR, "instruments_mc_125.csv"), sep=',')
premium_offset = 0. # 1 stands for 100% increase from the original premium values
simulation_length_days = 1000 # number of days
number_of_paths = 500 # number of the random paths to do backtest
number_of_utils = 100 # number of utils on the [0, 1] segment, passed to the get_kelly_curve
number_of_bins = np.linspace(-1, 5, 1000) # Number of bins. Could be also string (e.g. 'auto') or list of bin edges. Whatever complies with np.histogram
initial_util = 0.2
monte_carlo_paths = 1000
bonding_curve_resolution = 100
multiproc_cores = os.cpu_count() // 2 

In [ ]:
assets_info_dicts = input_df.to_dict(orient='records')


In [ ]:
multiproc_args_kelly_list = []

for asset_dict in assets_info_dicts:
    multiproc_args_kelly = [
        asset_dict.copy(),
        simulation_length_days,
        historical_df,
        monte_carlo_paths,
        bonding_curve_resolution,
        number_of_paths,
        0,
        "kelly",
        0,
    ]
    multiproc_args_kelly_list.append(multiproc_args_kelly)
# backtest each instrument from the assets_info_dicts
# and add the result to the res_assets_info_dicts
pool = multiprocessing.Pool(multiproc_cores)     
res_assets_info_dicts = pool.starmap(backtest.backtest_asset_dict, multiproc_args_kelly_list)


In [ ]:
# pack all the info the the DataFrame
summary_df = pd.DataFrame(res_assets_info_dicts)
# summary_df.sort_values(by='median_bankroll').head()

# unpack percentiles
percentile_df = pd.DataFrame(summary_df.max_drawdown_percentiles.tolist(), index= summary_df.index)
percentile_df.rename(columns=lambda col: f'{(0, 25, 50, 75, 100)[col]}_percentile_max_drawdown', inplace=True)
cagr_percentile_df = pd.DataFrame(summary_df.cagr_percentiles.tolist(), index= summary_df.index)
cagr_percentile_df.rename(columns=lambda col: f'{(0, 25, 50, 75, 100)[col]}_percentile_cagr', inplace=True)
summary_df = summary_df.merge(percentile_df, left_index=True, right_index=True)
summary_df = summary_df.merge(cagr_percentile_df, left_index=True, right_index=True)

In [ ]:
# Save results if needed
summary_df.drop(columns=['price_series', 'premium_vs_util_df', 'bankroll_df']).to_csv(os.path.join(hlp.OUTPUT_DATA_DIR, 'flow2_results_new.csv'))

In [ ]:
plotting.flow2_plot(
        summary_df,
        hue="option_type",
        cagr_pick=None,
        max_drawdown_pick=None,
        x_axis="100_percentile_max_drawdown",
        y_axis="50_percentile_cagr",
)

# Create instuments batches

In [ ]:

assets = [asset for asset in historical_df.columns if (not 'Unnam' in asset and not 'Master' in asset)]
strike_pcts = [i for i in range(-10, 10, 10)]
durations = [i for i in range(1, 31, 10)]
utilizations = [0.5]
train_start_dates = ['min']
train_end_dates = ['max']
moods = ['Full History'] # , 'bear', 'bull'

columns = ['asset', 'strike_pct', 'duration', 'util', 'train_start_date', 'train_end_date', 'mood']

all_combinations = list(itertools.product(assets, strike_pcts, durations, utilizations, train_start_dates, train_end_dates, moods))
instruments_df = pd.DataFrame(all_combinations, columns=columns)
instruments_df.to_csv(os.path.join(hlp.INPUT_DATA_DIR, 'flow2_input.csv'), index=False)

In [ ]:
print(instruments_df)